# Azure OpenAI Chat Completion Agent Demo

This notebook demonstrates how to use Azure OpenAI Chat Completion Agents with the Microsoft Agent Framework.

Based on: [Azure OpenAI Chat Completion Agents Documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types/azure-openai-chat-completion-agent?pivots=programming-language-python)

## Prerequisites

1. Azure CLI installed and authenticated (`az login`)
2. Azure OpenAI resource created with a deployed model
3. Environment variables set:
   - `AZURE_OPENAI_ENDPOINT`
   - `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
   - (Optional) `AZURE_OPENAI_API_VERSION`
   - (Optional) `AZURE_OPENAI_API_KEY` (if not using Azure CLI authentication)

## Setup: Load Environment Variables

First, we'll load the required environment variables from the `.env` file.

In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env file
# Navigate up from Notebooks -> agents -> getting_started -> samples -> python
env_path = "/Users/arturoquiroga/GITHUB/Agent-Framework-Private/python/.env"
load_dotenv(dotenv_path=env_path)

# Verify environment variables are loaded
print(f"OpenAI Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT', 'Not set')}")
print(f"Deployment Name: {os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'Not set')}")
print(f"API Version: {os.getenv('AZURE_OPENAI_API_VERSION', 'Using default')}")

OpenAI Endpoint: https://aq-ai-foundry-sweden-central.openai.azure.com/
Deployment Name: gpt-4.1
API Version: Using default


## Example 1: Basic Agent Creation

The simplest way to create an agent using environment variables.

In [5]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

async def basic_agent_example():
    """Create and use a basic Azure OpenAI Chat Completion agent."""
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="what is the square root of 878.",
        name="Joker"
    )

    result = await agent.run("what is the square root of 878.")
    print(f"Agent: {result.text}")

# Run the example
await basic_agent_example()

Agent: The square root of 878 is approximately **29.649**.

More precisely:  
√878 ≈ **29.6486**


## Example 2: Explicit Configuration

Instead of using environment variables, you can provide configuration explicitly.

In [6]:
async def explicit_config_example():
    """Create an agent with explicit configuration."""
    agent = AzureOpenAIChatClient(
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        deployment_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
        credential=AzureCliCredential()
    ).create_agent(
        instructions="You are a helpful math tutor.",
        name="MathTutor"
    )

    result = await agent.run("Explain the Pythagorean theorem in simple terms.")
    print(f"Agent: {result.text}")

# Run the example
await explicit_config_example()

Agent: Sure! The Pythagorean theorem is a rule about right-angled triangles (triangles with one 90° angle).

It says:

**If you have a right triangle, the square of the length of the longest side (called the hypotenuse) is equal to the sum of the squares of the other two sides.**

In math, it looks like this:
\[ a^2 + b^2 = c^2 \]

- **a** and **b** are the lengths of the two shorter sides,
- **c** is the length of the longest side, which is opposite the right angle.

**Example:**  
If one side is 3 and the other is 4, then:
\( 3^2 + 4^2 = 9 + 16 = 25 \)
So the hypotenuse is \( \sqrt{25} = 5 \).

The theorem helps you find the length of a side if you know the other two!


## Example 3: Streaming Responses

Get responses as they are generated using streaming for better user experience.

In [7]:
async def streaming_example():
    """Stream responses as they are generated."""
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="You are a creative storyteller.",
        name="StoryTeller"
    )

    print("Agent: ", end="", flush=True)
    async for chunk in agent.run_stream("Tell me a short story about a brave knight and a dragon."):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    print()  # New line at the end

# Run the example
await streaming_example()

Agent: Once upon a time in a misty green valley,Once upon a time in a misty green valley, there lived a brave knight named Sir Cedric. Clad in gleaming silver armor, Sir Cedric was known there lived a brave knight named Sir Cedric. Clad in gleaming silver armor, Sir Cedric was known across the land across the land for his unwavering courage and kind heart.

One crisp morning, word spread that a mighty dragon had taken residence atop nearby Emberfire Hill for his unwavering courage and kind heart.

One crisp morning, word spread that a mighty dragon had taken residence atop nearby Emberfire Hill. Fear gr. Fear gripped the villagers, for the beast's roar shook the windows and its shadow blotted out the sun.

Without hesitationipped the villagers, for the beast's roar shook the windows and its shadow blotted out the sun.

Without hesitation, Sir Cedric mounted his trusty horse, Sir Cedric mounted his trusty horse, Luna, and rode toward the hill. The climb was steep, the, Luna, and rode to

## Example 4: Function Tools - Single Function

Provide custom function tools to Azure OpenAI agents. Functions can be automatically called by the agent when needed.

In [8]:
from typing import Annotated
from pydantic import Field
import importlib.util
from pathlib import Path

# Import weather_utils from the azure_ai_real_weather directory (sibling directory)
weather_utils_path = Path("/Users/arturoquiroga/GITHUB/Agent-Framework-Private/python/samples/getting_started/agents") / "azure_ai_real_weather" / "weather_utils.py"
spec = importlib.util.spec_from_file_location("weather_utils", weather_utils_path)
weather_utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(weather_utils)
get_real_weather = weather_utils.get_real_weather

async def function_tool_example():
    """Use a custom function tool with an agent."""
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="You are a helpful assistant.",
        tools=get_real_weather
    )

    result = await agent.run("What is the weather like in Amsterdam?")
    print(f"Agent: {result.text}")

# Run the example
await function_tool_example()

Agent: The current weather in Amsterdam is clear sky, with a temperature of 17.3°C (feels like 16.7°C). The humidity is 63%, and the wind speed is 6.17 m/s.


## Example 5: Function Tools with Decorator

Use the `@ai_function` decorator to explicitly specify function name and description.

In [12]:
from agent_framework import ai_function
import importlib.util
from pathlib import Path

# Import weather_utils from the azure_ai_real_weather directory (sibling directory)
weather_utils_path = Path("/Users/arturoquiroga/GITHUB/Agent-Framework-Private/python/samples/getting_started/agents") / "azure_ai_real_weather" / "weather_utils.py"
spec = importlib.util.spec_from_file_location("weather_utils", weather_utils_path)
weather_utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(weather_utils)
get_real_weather = weather_utils.get_real_weather

# Decorate the get_real_weather function
@ai_function(name="weather_tool", description="Retrieves weather information for any location")
def get_weather_decorated(*args, **kwargs):
    return get_real_weather(*args, **kwargs)

async def decorated_function_example():
    """Use a decorated function tool."""
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="You are a helpful assistant.",
        tools=get_weather_decorated
    )

    result = await agent.run("What's the weather like in Tokyo?")
    print(f"Agent: {result.text}")

# Run the example
await decorated_function_example()

ValidationError: 1 validation error for weather_tool_input
kwargs
  Field required [type=missing, input_value={'chat_options': <agent_f...5db50>, 'args': 'Tokyo'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

## Example 6: Multiple Function Tools

Provide multiple function tools to an agent.

In [14]:
from random import randint
from datetime import datetime

def get_current_time() -> str:
    """Get the current time."""
    return f"The current time is {datetime.now().strftime('%H:%M:%S')}."

def roll_dice(
    sides: Annotated[int, Field(description="Number of sides on the dice", ge=4, le=100)] = 6
) -> str:
    """Roll a dice with the specified number of sides."""
    result = randint(1, sides)
    return f"You rolled a {result} on a {sides}-sided dice!"

def calculate_tip(
    bill_amount: Annotated[float, Field(description="The total bill amount")],
    tip_percentage: Annotated[float, Field(description="Tip percentage (e.g., 15 for 15%)")]
) -> str:
    """Calculate the tip amount and total bill."""
    tip_amount = bill_amount * (tip_percentage / 100)
    total = bill_amount + tip_amount
    return f"Tip amount: ${tip_amount:.2f}, Total: ${total:.2f}"

async def multiple_tools_example():
    """Use multiple function tools together."""
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="You are a helpful assistant.",
        tools=[get_current_time, roll_dice, calculate_tip, get_real_weather]
    )

    # Query that might use multiple tools
    result = await agent.run(
        "First, tell me the current time. Then roll a 20-sided dice. "
        "Finally, calculate a 18% tip on a $75.50 bill."
    )
    print(f"Agent: {result.text}")

# Run the example
await multiple_tools_example()

Agent: Here are your requested results:

1. The current time is 10:41:39.
2. You rolled an 18 on a 20-sided dice!
3. An 18% tip on a $75.50 bill is $13.59, making the total $89.09.


## Example 7: Function Tools as a Class

Organize related functions together in a class. This is useful for sharing state between functions.

In [15]:
class WeatherTools:
    """A collection of weather-related tools."""
    
    def __init__(self):
        # Simulated weather cache
        self.weather_cache = {
            "Seattle": "Rainy, 12°C",
            "Miami": "Sunny, 28°C",
            "London": "Cloudy, 15°C",
            "Sydney": "Partly cloudy, 22°C"
        }
    
    def get_weather(self, location: Annotated[str, Field(description="The location to get the weather for.")]) -> str:
        """Get the current weather for a location."""
        return f"The weather in {location} is {self.weather_cache.get(location, 'sunny with a high of 20°C')}."
    
    def get_weather_details(
        self,
        location: Annotated[str, Field(description="The location to get detailed weather for.")]
    ) -> str:
        """Get detailed weather information including forecast."""
        base_weather = self.weather_cache.get(location, "sunny, 20°C")
        return f"Current weather in {location}: {base_weather}. Tomorrow's forecast: Similar conditions expected."

async def class_tools_example():
    """Use function tools from a class."""
    tools = WeatherTools()
    
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="You are a helpful weather assistant.",
        tools=[tools.get_weather, tools.get_weather_details]
    )

    result = await agent.run("What's the weather like in Miami? Also give me the detailed forecast.")
    print(f"Agent: {result.text}")

# Run the example
await class_tools_example()

Agent: The current weather in Miami is sunny with a temperature of 28°C.

The detailed forecast for tomorrow in Miami shows similar sunny conditions are expected.


## Example 8: Conversation with Memory (Using ChatAgent)

Use `ChatAgent` wrapper for more control and conversation memory across multiple turns.

In [22]:
from agent_framework import ChatAgent

async def conversation_memory_example():
    """Demonstrate conversation memory with multiple turns."""
    # Create a chat client
    chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())
    
    # Create a chat agent with the client
    agent = ChatAgent(
        chat_client=chat_client,
        name="MemoryAgent",
        instructions="You are a helpful assistant that remembers conversation context.",
        tools=get_real_weather
    )
    
    # Create a thread to maintain conversation history
    thread = agent.get_new_thread()
    
    # First turn
    print("User: My name is Alex and I'm planning a trip to Paris.")
    result1 = await agent.run(
        "My name is Alex and I'm planning a trip to Paris.",
        thread=thread,
        store=True
    )
    print(f"Agent: {result1.text}\n")
    
    # Second turn - agent should remember the name and destination
    print("User: What's my name and where am I going?")
    result2 = await agent.run(
        "What's my name and where am I going?",
        thread=thread,
        store=True
    )
    print(f"Agent: {result2.text}\n")
    
    # Third turn - check weather using the tool
    print("User: Can you check the weather for my destination?")
    result3 = await agent.run(
        "Can you check the weather for my destination?",
        thread=thread,
        store=True
    )
    print(f"Agent: {result3.text}")

# Run the example
await conversation_memory_example()

User: My name is Alex and I'm planning a trip to Paris.
Agent: Nice to meet you, Alex! Paris is an incredible destination with so much to see and do. How can I assist you with your trip planning? Are you looking for things to do, travel tips, hotel or restaurant recommendations, or perhaps current weather information? Let me know how I can help make your trip amazing!

User: What's my name and where am I going?
Agent: Nice to meet you, Alex! Paris is an incredible destination with so much to see and do. How can I assist you with your trip planning? Are you looking for things to do, travel tips, hotel or restaurant recommendations, or perhaps current weather information? Let me know how I can help make your trip amazing!

User: What's my name and where am I going?
Agent: Your name is Alex, and you are planning a trip to Paris. If you need help with any details or have any questions about your trip, just let me know!

User: Can you check the weather for my destination?
Agent: Your name i

ServiceResponseException: <class 'agent_framework.azure._chat_client.AzureOpenAIChatClient'> service failed to complete the prompt: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}

## Example 9: Streaming with Function Calls

Combine streaming responses with function tool calls.

In [21]:
async def streaming_with_tools_example():
    """Stream responses while using function tools."""
    agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
        instructions="You are a helpful assistant. When providing weather information, always use the available tool.",
        tools=[get_real_weather, roll_dice]
    )

    print("Agent: ", end="", flush=True)
    async for chunk in agent.run_stream(
        "Check the weather in Seattle, then roll a dice and write a short poem about both results."
    ):
        if chunk.text:
            print(chunk.text, end="", flush=True)
    print()  # New line at the end

# Run the example
await streaming_with_tools_example()

Agent: 

Here's a short poem inspired by today's events:

In Seattle's sky, a short poem inspired by today's events:

In Seattle's sky, the clouds all roam,
Broken and drifting, far from home.
A the clouds all roam,
Broken and drifting, far from home.
A gentle breeze, cool and profound gentle breeze, cool and profound,
While temperatures hover, calm all around.

A dice was rolled, and luck,
While temperatures hover, calm all around.

A dice was rolled, and luck did mix,
It spun and landed straight on six!
Weather or did mix,
It spun and landed straight on six!
Weather or chance, both have their chance, both have their say,
Bringing a little magic to this day.
 say,
Bringing a little magic to this day.


## Example 10: Error Handling

Demonstrate proper error handling when working with agents.

In [19]:
async def error_handling_example():
    """Demonstrate error handling with agents."""
    try:
        agent = AzureOpenAIChatClient(credential=AzureCliCredential()).create_agent(
            instructions="You are a helpful assistant.",
            name="ErrorDemo"
        )
        
        result = await agent.run("Hello, how are you?")
        print(f"Success: {result.text}")
        
    except Exception as e:
        print(f"Error occurred: {type(e).__name__}: {str(e)}")
        print("\nMake sure your environment variables are set correctly:")
        print("- AZURE_OPENAI_ENDPOINT")
        print("- AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
        print("\nAnd that you've run 'az login' for authentication.")

# Run the example
await error_handling_example()

Success: Hello! I’m just a computer program, but I’m here and ready to help you. How can I assist you today?


## Example 11: Advanced - Custom Chat Client Configuration

Configure the chat client with additional parameters like temperature, max tokens, etc.

In [18]:
async def advanced_config_example():
    """Use advanced configuration options."""
    # Create a chat client with custom settings
    chat_client = AzureOpenAIChatClient(
        endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        deployment_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
        credential=AzureCliCredential(),
        # Note: Additional parameters like temperature are typically set per-request
        # rather than on the client itself
    )
    
    agent = chat_client.create_agent(
        instructions="You are a creative writer. Be very creative and imaginative.",
        name="CreativeWriter"
    )

    result = await agent.run("Write a one-sentence description of a futuristic city.")
    print(f"Agent: {result.text}")

# Run the example
await advanced_config_example()

Agent: Suspended amid clouds of neon mist, the city spirals skyward on living glass tendrils, where bioluminescent gardens bloom upon gravity-defying terraces and sentient drones sing lullabies through the ever-shifting, holographic skyline.


## Summary

This notebook demonstrated key features of Azure OpenAI Chat Completion Agents:

1. **Basic Agent Creation** - Simple agent setup with environment variables
2. **Explicit Configuration** - Direct configuration without environment variables
3. **Streaming Responses** - Real-time response generation
4. **Single Function Tool** - Custom function as a tool
5. **Decorated Functions** - Using `@ai_function` decorator
6. **Multiple Function Tools** - Combining multiple tools
7. **Class-based Tools** - Organizing related functions in a class
8. **Conversation Memory** - Maintaining context with ChatAgent and threads
9. **Streaming with Tools** - Combining streaming and function calls
10. **Error Handling** - Proper exception handling
11. **Advanced Configuration** - Custom client settings

## Key Differences from Azure AI Foundry Agents

- **No persistent storage**: Chat completion agents don't persist to a service
- **Custom history storage**: You manage conversation history yourself
- **More flexible**: Direct access to Azure OpenAI Chat Completion API
- **Lighter weight**: No need for Azure AI Foundry project setup

## Next Steps

- Explore [Agent Framework Tutorials](https://learn.microsoft.com/en-us/agent-framework/tutorials/overview)
- Learn about [Azure AI Foundry Agents](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types/azure-ai-foundry-agent)
- Try [OpenAI Chat Completion Agents](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types/openai-chat-completion-agent)
- Check out more examples in the `samples` directory